# Baby Food Heavy Metals
### *Part 1 - Scrapping Appendix A*

In October 2019 Healthy Babies Bright Futures came out with a report showing that Toxic Heavy Metals were found in nearly every baby food they tested [link](https://www.healthybabyfood.org/sites/healthybabyfoods.org/files/2020-04/BabyFoodReport_ENGLISH_R6.pdf) (n=168).  Their overall guidance from the report was excellent; but I had some questions I wanted to answer myself.  In order to do this I needed a workable table of their laboratory results.  

In this Script I scrape and clean up Appendix A from Healthy Babies Bright Futures report, 'What's in my Baby's Food?'.

In [1]:
import numpy as np
import pandas as pd
import tabula

##### Here is the link of the PDF online.

In [2]:
file = "https://www.healthybabyfood.org/sites/healthybabyfoods.org/files/2020-04/BabyFoodReport_ENGLISH_R6.pdf"

##### Appendix A is pages 22 to 31.   I'm having trouble reading multiple pages with tabula in this particular pdf.  TTo resolve this I  in one data frame, although when I run this on individual pages its not a problem.  May need to create a loop, and comcat this to one giant dataframe.  Easiest would be to do this first.

In [3]:
Tables = pd.DataFrame()
Tables_stack = pd.DataFrame()

In [4]:
for i in range(22, 32):
    Tables = tabula.read_pdf(file, pages = [i], stream=True)[0]
    Tables_stack = pd.concat([Tables_stack, Tables], axis=0)

In [5]:
Tables_stack = Tables_stack.replace(np.nan, '', regex=True)

##### Columns had issues due to text wrapping over 3 lines.  The table is justified to the bottom, so only columns with 3 lines have any phrases in the dataframe column name.  To solve this a for loop was ran around  dataframe of only the first two rows.  This creates a_array, which I then copied and pasted and given minor edits into a numpy called Col; then defined as the main dataframes column labels.

In [6]:
df = Tables_stack.head(2)

In [7]:
a_array = np.array([])

In [8]:
for j in df.columns: # j is the columns, i is the rows
    if "Unnamed:" in j:
        a = (df[j].str.cat())
    else:
        a = (j + df[j].str.cat(sep=' '))
    print(a)
    a_array = np.append(a_array, a)

Brand Food

Food type
Arsenic(total, ppb)
Arsenic(inorganic, ppb)
Lead (ppb)
Cadmium(ppb)
Mercury(total, ppb)
Metro area wherepurchased
Retailer


In [9]:
a_array

array(['Brand Food', '', 'Food type', 'Arsenic(total, ppb)',
       'Arsenic(inorganic, ppb)', 'Lead (ppb)', 'Cadmium(ppb)',
       'Mercury(total, ppb)', 'Metro area wherepurchased', 'Retailer'],
      dtype='<U32')

In [10]:
Col = ['Brand', 'Food', 'Food type', 'Arsenic (total, ppb)',
       'Arsenic (inorganic, ppb)', 'Lead (ppb)', 'Cadmium (ppb)',
       'Mercury (total, ppb)', 'Metro area where purchased', 'Retailer']

In [11]:
Tables_stack.columns = Col

In [12]:
Tables_stack.shape

(351, 10)

##### We want to drop the column names repeated on every page; 2 rows repeated 10x (once for every page).  The first two rows(0 & 1) are dropped, since we concatted the tables the same number index is used reducing our overall rows by 20 (351 to 331).

In [13]:
Tables_stack = Tables_stack.drop([0,1])
Tables_stack.shape

(331, 10)

In [14]:
Tables_stack = Tables_stack.reset_index(drop=True)

##### Each food tested has multiple rows page has different seperation based on the wrapping of the Brand, Food and Food Type columns. However it always begins on a specific value or in this case a specific Retailer.  To fix this a new column is created 'a', which counts everytime a new Retailer appears.  Then, columns Brand, Food & Food Type are tranformed to 'join', grouping by the 'a' value.

##### Then all blank values are replaced with NaN, and any row in 'Retailer' that has NaN is removed.  After the column 'a' is dropped.

In [15]:
a_array = np.array([]) 
for index, row in Tables_stack.iterrows():
    if row.Retailer != '':
        a = int(index)
    a_array = np.append(a_array, a)
Tables_stack['a']= a_array
Tables_stack['Brand'] = Tables_stack.groupby(['a'])['Brand'].transform(lambda x: ' '.join(x))
Tables_stack['Food'] = Tables_stack.groupby(['a'])['Food'].transform(lambda y: ' '.join(y))
Tables_stack['Food type'] = Tables_stack.groupby(['a'])['Food type'].transform(lambda z: ' '.join(z))
Tables_stack.replace("", np.nan, inplace=True) # replace blank values with NaN
Tables_stack = Tables_stack.dropna(subset = ['Retailer']) #Drop rows with NaN in Values
Tables_stack = Tables_stack.drop(columns=['a'])

In [16]:
Tables_stack.shape

(168, 10)

##### Based on the shape shoing 168 rows, it looks like this method worked as the article tested 168 unique baby foods.

##### After this there are some minor things to do to cleanup the table further to make it easy to work with.

In [17]:
Tables_stack.replace(" ", np.nan, inplace=True)
Tables_stack.replace("  ", np.nan, inplace=True)
Tables_stack.replace("   ", np.nan, inplace=True)

##### In some cases the Food column was empty, and the Brand column included the Brand & Food information.  To fix this every Food value that has 'NaN' in it will be replaced by the entire Brand column.  This will result in a Food that has the Brand preceding its name.  This is minor difference in the tables ouput from this script and the original pdf.

In [18]:
Tables_stack.Food.fillna(Tables_stack.Brand, inplace=True)

##### To remove all the extra text in Brand, first only the first word is kept to use to identify the entire Brand (45 unique brands).  Then an array, mult, was created with brute force (typing in each brand name).  A dataframe from this array, df, is created where one column 'First' has only the first word, and 'mult' represents multiple words for the brand.  This dataframe is then used to create a dictionary which then replaces each identical term to the 'first' word with the 'mult' column.

In [19]:
Tables_stack['Brand'] = Tables_stack['Brand'].str.split().str[0].str.strip()

In [20]:
Tables_stack['Brand'].unique()

array(['Beech-Nut', 'BioKinetics', 'Earth’s', 'Gerber', 'Healthy',
       'Kitchdee', 'HappyBABY', 'Harvest', 'Cream', 'NurturMe', '365',
       'Baby’s', 'Enfamil', 'Meijer', 'Parent’s', 'Plum', 'Similac',
       'Simple', 'up', 'O', 'Applesnax', 'Mott’s', 'Seneca', 'Orchard',
       'Sprout', 'Juicy', 'Kidgets', 'Apple', 'Good2Grow', 'Orgain',
       'Pediasure', 'Repone', 'Yoo-hoo', 'Deluxe', 'Happy', 'Kraft',
       'Comforts', 'Baby', 'Cuétara', 'Lil’', 'Nosh!', 'Ella’s', 'Little',
       'Nostalgia', 'SOBISK'], dtype=object)

In [21]:
Tables_stack['Brand'].unique().size

45

In [22]:
mult = np.array(['Earth’s Best', 'Healthy Times', 'Kitchdee Organic', 'Harvest Hill', 'Cream of Wheat',
                 '365 organic(Whole Foods)', 'Baby’s Only Organic', 'Parent’s Choice (Walmart)', 'Plum Organics', 
                 'Simple Truth Organic (Kroger)','up & up (Target)', 'O Organics (Albertson/Safeway)','Orchard Naturals',
                 'Apple & Eve','Juicy Juice','Deluxe Pasta','Happy Tot','Baby Mum-Mum', 'Simple Truth Organic (Kroger)', 
                 'Lil’ Dutch Maid', 'Ella’s Kitchen', 'Little Duck Organics', 'Beech-Nut', 'BioKinetics', 'Gerber', 
                 'HappyBABY', 'NurturMe', 'Enfamil', 'Meijer', 'Similac', 'Applesnax', 'Mott’s', 'Seneca', 'Sprout', 
                 'Kidgets', 'Good2Grow', 'Orgain', 'Pediasure', 'Repone', 'Yoo-hoo', 'Kraft', 'Comforts (Kroger)', 
                 'Cuétara', 'Nosh!', 'Nostalgia', 'SOBISK'])

In [23]:
df = pd.DataFrame([])
df['mult']=mult
df['First'] = df['mult'].str.split().str[0].str.strip()
df.head()

,mult,First
0,Earth’s Best,Earth’s
1,Healthy Times,Healthy
2,Kitchdee Organic,Kitchdee
3,Harvest Hill,Harvest
4,Cream of Wheat,Cream


In [24]:
key_list = list(Tables_stack['Brand'])

In [25]:
dict_lookup = dict(zip(df['First'], df['mult']))

In [26]:
Tables_stack['Brand'] = [dict_lookup[item] for item in key_list]

In [27]:
Tables_stack = Tables_stack.reset_index(drop=True)

In [28]:
Tables_stack.columns

Index(['Brand', 'Food', 'Food type', 'Arsenic (total, ppb)',
       'Arsenic (inorganic, ppb)', 'Lead (ppb)', 'Cadmium (ppb)',
       'Mercury (total, ppb)', 'Metro area where purchased', 'Retailer'],
      dtype='object')

##### Checking that everything looks right, then removing a unique character that causes an issue on excel.  The character is  ’  which is cleaner when replaced with '.

In [29]:
Tables_stack.head()

,Brand,Food,Food type,"Arsenic (total, ppb)","Arsenic (inorganic, ppb)",Lead (ppb),Cadmium (ppb),"Mercury (total, ppb)",Metro area where purchased,Retailer
0,Beech-Nut,Beech-Nut Rice Single Grain Baby Cereal - Stag...,Cereal - rice,117,86,3.5,5.4,0.582,"Charlottesville, VA",Wegmans
1,BioKinetics,BioKinetics BioKinetics Brown Rice Organic Spr...,Cereal - rice,353,144,3.1 *,31.7,2.32,"Washington, DC",amazon.com
2,Earth’s Best,Earth’s Best Whole Grain Rice Cereal,Cereal - rice,138,113,22.5,14.7,2.41,"San Diego, CA",99 Cents Only Stores
3,Earth’s Best,Earth’s Best Whole Grain Rice Cereal,Cereal - rice,126,107,17.8,13.4,2.19,"Portland, ME",Hannaford
4,Gerber,Gerber Rice Single Grain Cereal,Cereal - rice,106,74,3.9,11.1,1.79,"Gambell, AK",ANICA Native Store


In [30]:
Tables_stack['Brand'] = Tables_stack['Brand'].str.replace('’','\'')
Tables_stack['Food'] = Tables_stack['Food'].str.replace('’','\'')

##### The CSV output 'Baby_Metals_clean_table.csv' is used to allow for future edits.  For now I have my own edits that I will make and put into an alternate CSV.

In [31]:
Tables_stack.to_csv('Baby_Metals_clean_table.csv') 

##### The alternate CSV is entiteld 'Baby_Metals_mod_data.csv' as it includes modified data.  The changes are:

**Cosmetic:**
- replacing '--' with blank.  -- means no test was performed.
- replacing 'é' with e.  This is only for one Brand/food name.
- I'm unsure of the reason, one data point has a ' v'; replaced with a blank.

**Data assumptions:**
- numerical values with < symbol are changed to 0.  This indicates there was no detection, but the value there is the limit of detection.  For our purposes this can be assumed at 0.
- The single * indicates the value is an estimate.  To give it a fair comparison, we assume that the estimate is accurate.
- ‡ is removed.  It's there to indicate this particular sample is the average of 3 data points.  Instead we will just keep the average.


In [32]:
Tables_stack = Tables_stack.replace('--','')
Tables_stack['Brand'] = Tables_stack['Brand'].str.replace("é","e")
Tables_stack['Food'] = Tables_stack['Food'].str.replace("é","e")

In [33]:
Tables_stack.loc[Tables_stack['Arsenic (total, ppb)'].str.contains('<'), 'Arsenic (total, ppb)'] = '0'
Tables_stack.loc[Tables_stack['Arsenic (inorganic, ppb)'].str.contains('<'), 'Arsenic (inorganic, ppb)'] = '0'
Tables_stack.loc[Tables_stack['Lead (ppb)'].str.contains('<'), 'Lead (ppb)'] = '0'
Tables_stack.loc[Tables_stack['Cadmium (ppb)'].str.contains('<'), 'Cadmium (ppb)'] = '0'
Tables_stack.loc[Tables_stack['Mercury (total, ppb)'].str.contains('<'), 'Mercury (total, ppb)'] = '0'

In [34]:
Tables_stack['Arsenic (total, ppb)'] = Tables_stack['Arsenic (total, ppb)'].str.replace("\*","")
Tables_stack['Arsenic (total, ppb)'] = Tables_stack['Arsenic (total, ppb)'].str.replace("\‡","")
Tables_stack['Lead (ppb)'] = Tables_stack['Lead (ppb)'].str.replace(" v","")
Tables_stack['Arsenic (inorganic, ppb)'] = Tables_stack['Arsenic (inorganic, ppb)'].str.replace("\*","")
Tables_stack['Lead (ppb)'] = Tables_stack['Lead (ppb)'].str.replace("\*","")
Tables_stack['Cadmium (ppb)'] = Tables_stack['Cadmium (ppb)'].str.replace("\*","")
Tables_stack['Mercury (total, ppb)'] = Tables_stack['Mercury (total, ppb)'].str.replace("\*","")

##### Output to 'Baby_Metals_mod_data.csv' and Checked that everything looks reasonable.

In [35]:
Tables_stack.to_csv('Baby_Metals_mod_data.csv') 

In [36]:
Tables_stack.head(20)

,Brand,Food,Food type,"Arsenic (total, ppb)","Arsenic (inorganic, ppb)",Lead (ppb),Cadmium (ppb),"Mercury (total, ppb)",Metro area where purchased,Retailer
0,Beech-Nut,Beech-Nut Rice Single Grain Baby Cereal - Stag...,Cereal - rice,117,86,3.5,5.4,0.582,"Charlottesville, VA",Wegmans
1,BioKinetics,BioKinetics BioKinetics Brown Rice Organic Spr...,Cereal - rice,353,144,3.1,31.7,2.32,"Washington, DC",amazon.com
2,Earth's Best,Earth's Best Whole Grain Rice Cereal,Cereal - rice,138,113,22.5,14.7,2.41,"San Diego, CA",99 Cents Only Stores
3,Earth's Best,Earth's Best Whole Grain Rice Cereal,Cereal - rice,126,107,17.8,13.4,2.19,"Portland, ME",Hannaford
4,Gerber,Gerber Rice Single Grain Cereal,Cereal - rice,106,74,3.9,11.1,1.79,"Gambell, AK",ANICA Native Store
5,Healthy Times,Healthy Times Organic Brown Rice Cereal - 4+ m...,Cereal - rice,153,133,67.4,12.1,1.53,"Washington, DC",amazon.com
6,Kitchdee Organic,Kitchdee Organic Baby Cereal Rice and Lentil -...,Cereal - rice,79.3,78,10.9,13.1,4.06,"Washington, DC",amazon.com
7,Gerber,Gerber MultiGrain Cereal - Sitter 2nd Foods,Cereal - mixed and multi-grain,37,31,5.3,26.2,0.367,"Detroit, MI",Meijer
8,HappyBABY,HappyBABY Oats & Quinoa Baby Cereal Organic Wh...,Cereal - mixed and multi-grain,10.2,,0.9,12.4,0,"Minneapolis, MN",Target
9,Beech-Nut,Beech-Nut Oatmeal Whole Grain Baby Cereal - St...,Cereal - oatmeal,23.8,,2.2,13,0,"Portland, OR",Fred Meyer
